<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Notebook-made-by" data-toc-modified-id="Notebook-made-by-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Notebook made by</a></span></li><li><span><a href="#Search-Engine" data-toc-modified-id="Search-Engine-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Search Engine</a></span></li><li><span><a href="#Assignment" data-toc-modified-id="Assignment-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Assignment</a></span><ul class="toc-item"><li><span><a href="#Requirement-1" data-toc-modified-id="Requirement-1-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Requirement 1</a></span></li><li><span><a href="#Requirement-2" data-toc-modified-id="Requirement-2-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Requirement 2</a></span></li><li><span><a href="#Requirement-3" data-toc-modified-id="Requirement-3-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Requirement 3</a></span></li><li><span><a href="#Requirement-4" data-toc-modified-id="Requirement-4-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Requirement 4</a></span></li><li><span><a href="#Requirement-5" data-toc-modified-id="Requirement-5-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Requirement 5</a></span></li><li><span><a href="#Requirement-6" data-toc-modified-id="Requirement-6-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Requirement 6</a></span></li></ul></li></ul></div>

# Notebook made by  

__Name(s)__: Chandni Bagchi, Judith Corsel, Magda Pannekoek, Nina Verheijen

__Student id(s)__ : 11824883, 11891173, 11621958, 11764198

 

# Search Engine

first off, all the code of the search engine

In [10]:
""" import packages """
from bs4 import BeautifulSoup
import nltk
import pandas as pd
import numpy as np

import xml.etree.ElementTree as ET
import re
from collections import defaultdict
from tqdm import tqdm_notebook
import ipywidgets as widgets
import zipfile
from IPython.core.display import display, HTML
import json
import webbrowser
import ipywidgets as widgets
import pandas as pd
from datetime import datetime
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from elasticsearch import Elasticsearch

import matplotlib.pyplot as plt
%matplotlib inline

#List of Dutch stopwords
stopwords = ["de", "het", "zijn", "niet", "heeft", "en", "dat", "hij", "zij", "het", "van", "in", "uit", "ook", "die", "wat", "te",
            "is", "een", "had", "vanaf", "voor", "op", "zich", "met", "naar", "bij", "aan", "door", "als", "tot", "of", "geen", "wel", "ecli"]


AttributeError: module 'html5lib.treebuilders' has no attribute '_base'

In [11]:
""" process one zipfile """
# Process an opened file, and retrieve all necessary values to put into the dictionary.
def process_file(file):
    tree = ET.parse(file)
    root = tree.getroot()

    # Get all tags with corresponding text.
    attrs = []
    texts = []
    counter = 0
    for child in root.iter():
        counter += 1
        tag = child.tag
        curlplace = tag.find('}')
        attr = tag[curlplace+1:]
        attrs.append(attr)
        texts.append(child.text)
        if counter > 80:
            break

    # Create dataframe with xml data.
    df = pd.DataFrame({'tags': attrs,'text': texts})

    # Check if document contains inhoudsindicatie.
    inh = get_inhoud(df)
    inst = get_instantie(df)
    recht = get_rechtsgebied(df)
    return inh, inst, recht

# Get inhoudsindicatie (a small summary from the 'rechtspraak') from the xml file.
def get_inhoud(df):
    inhoud_str = ''
    if df['tags'].str.contains('inhoudsindicatie').any():
        # Select only the inhoudsindicatie.
        selection = df[df.tags == 'inhoudsindicatie'].index.values[0]
        inhoud_df = df.truncate(before = selection)
        selection = df[df.tags == 'uitspraak']
        if selection.empty:
            selection = df[df.tags == 'conclusie']
        inhoud_df = inhoud_df.truncate(after = selection.index.values[0])

        # Add only the parts containing letters, and skip ones that are just spaces and enters.
        for line in inhoud_df.text:
            if line is None:
                continue
            if inhoud_str[-1:] != '.' and inhoud_str != '':
                inhoud_str += '.'
            if re.search('[a-zA-Z]', line):
                inhoud_str += ' ' + str(line)
        inhoud_str = inhoud_str.replace('\n', '')
        return inhoud_str
    
def proces_inhoud(inhoud_dict, string, ecli):
    for word in string.split(' '):
        if word.isalpha():
            inhoud_dict[word] += ecli + ' '
    return inhoud_dict

# Filter for 'instantie' (instance).
def get_instantie(df):
    if not df[df.tags == 'creator'].empty:
        return df[df.tags == 'creator']['text'].iloc[0]
    return None

# Filter for 'rechtsgebied' (jurisdiction).
def get_rechtsgebied(df):
    if not df[df.tags == 'subject'].empty:
        return df[df.tags == 'subject']['text'].iloc[0]
    return None

# Access zipfolder.
def process_folder(zipfolder): 
    archive = zipfile.ZipFile(zipfolder, 'r')
    namelist = [x for x in archive.namelist() if '.xml' in x and x[0] is not 'E']
    year = int(zipfolder[:4])
    month = int(zipfolder[4:6])
    inhoud_dict = defaultdict(str)
    lawsuit_dict = defaultdict(lambda: dict())
    
    
    # Go through each file in zipfolder.
    for infile in tqdm_notebook(namelist):
        ecli = infile[7:-4]
        f = archive.open(infile)
        
        # Extract information from file.
        inhoud, instantie, rechtsgebied = process_file(f)
        
        if inhoud:
            inhoud_dict = proces_inhoud(inhoud_dict, inhoud, ecli)
        
        # Fill dictionary with the values.
        lawsuit_dict[ecli]['year'] = year
        lawsuit_dict[ecli]['month'] = month
        lawsuit_dict[ecli]['inhoud'] = inhoud
        lawsuit_dict[ecli]['instantie'] = instantie
        lawsuit_dict[ecli]['rechtsgebied'] = rechtsgebied
        lawsuit_dict[ecli]['ecli'] = ecli

    return lawsuit_dict, inhoud_dict


zipfolder = '201903.zip'
dict1903 = process_folder(zipfolder)

NameError: name 'zipfile' is not defined

In [12]:
""" delete indices if necessary """
# e.indices.delete(index = 'rechtzaken')
# e.indices.delete(index = 'keywords')

' delete indices if necessary '

In [13]:
""" create indices if necessary """
e = Elasticsearch()
# e.indices.create('keywords')
# e.indices.create('rechtzaken')

NameError: name 'Elasticsearch' is not defined

In [14]:
""" import data into elasticsearch if necessary """
def rec_to_actions(df):
    import json
    for record in df.to_dict(orient="records"):
        yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX, TYPE))
        yield (json.dumps(record, default=int))
        
# INDEX = 'rechtzaken'
# TYPE = 'record'
# df_dict_rechtzaken = (pd.DataFrame.from_dict(dict(dict1903[0]), orient='index'))
# res = e.bulk(rec_to_actions(df_dict_rechtzaken), request_timeout=10)
# INDEX = 'keywords'
# df_dict_keywords = (pd.DataFrame.from_dict(dict(dict1903[1]), orient='index'))
# res = e.bulk(rec_to_actions(df_dict_keywords), request_timeout=10)

In [15]:
""" helper function to get url to rechspraak site from ecli number of rechtzaak """
def get_url(ecli, text):
    ecli = ecli.replace('_', ':')
    url = 'https://uitspraken.rechtspraak.nl/inziendocument?id={}'.format(ecli)
    return display(HTML('<a href="{}", target = blank, style="color: #000000; text-decoration: none;">{}</a>'.format(url, text)))


In [16]:
""" create widgets for the faceted search """
# year slider
sliderYear = widgets.IntRangeSlider(
    value=[1913, 2019],
    min=1913,
    max=2019,
    step=1,
    description='Jaar:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

# month slider
sliderMonth = widgets.IntRangeSlider(
    value=[1, 12],
    min=1,
    max=12,
    step=1,
    description='Maand:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

# rechtsgebied selector
selector = widgets.SelectMultiple(
    options=['Bestuursrecht', 'Civiel recht', 'Internationaal publiekrecht', 'Strafrecht'],
    description='Rechtsgebied',
    disabled=False
)

# instantie selector
selector2 = widgets.SelectMultiple(
    options=["Kamer voor het notariaat in het ressort 's-Hertogenbosch", 'Rechtbank Overijssel', 'Parket bij de Hoge Raad', 'Gerecht in eerste aanleg van Sint Maarten', 'Gerecht in Eerste Aanleg van Aruba', 'College van Beroep voor het bedrijfsleven', 'Gerecht in eerste aanleg van Bonaire, Sint Eustatius en Saba', "Regionaal Tuchtcollege voor de Gezondheidszorg 's-Gravenhage", 'College voor de Rechten van de Mens', 'Gemeenschappelijk Hof van Justitie van Aruba, Curaçao, Sint Maarten en van Bonaire, Sint Eustatius en Saba', 'Kamer voor het notariaat in het ressort Amsterdam', 'Hof van Discipline', 'Rechtbank Gelderland', 'Accountantskamer NIVRA', 'Centraal Tuchtcollege voor de Gezondheidszorg', 'Kamer voor het notariaat in het ressort Arnhem-Leeuwarden', 'Raad van State', 'Rechtbank Zeeland-West-Brabant', "Raad van Discipline 's-Gravenhage", 'Regionaal Tuchtcollege voor de Gezondheidszorg Amsterdam', 'Rechtbank Limburg', 'Gerechtshof Amsterdam', "Raad van Discipline 's-Hertogenbosch", 'Regionaal Tuchtcollege voor de Gezondheidszorg Zwolle', 'Kamer voor het notariaat in het ressort Den Haag', 'Rechtbank Den Haag', 'Rechtbank Midden-Nederland', 'Rechtbank Noord-Nederland', 'Regionaal Tuchtcollege voor de Gezondheidszorg Eindhoven', 'Rechtbank Amsterdam', 'Hoge Raad', 'Kamer voor Gerechtsdeurwaarders', 'Gerecht in eerste aanleg van Curaçao', 'Raad van Discipline Arnhem-Leeuwarden', 'Gerechtshof Arnhem-Leeuwarden', 'Rechtbank Oost-Brabant', "Gerechtshof 's-Hertogenbosch", 'Rechtbank Noord-Holland', 'Raad van Discipline Amsterdam', 'Gerecht in Ambtenarenzaken van Aruba, Curaçao, Sint Maarten en van Bonaire, Sint Eustatius en Saba', 'Rechtbank Rotterdam', 'Regionaal Tuchtcollege voor de Gezondheidszorg Groningen', 'Gerechtshof Den Haag', 'Centrale Raad van Beroep'],
    rows=8,
    description='Instantie',
    disabled=False
)


NameError: name 'widgets' is not defined

In [17]:
""" display faceted search UI """
accordion = widgets.Accordion(children=[sliderYear, sliderMonth, selector, selector2])
accordion.set_title(0, 'Jaar')
accordion.set_title(1, 'Maand')
accordion.set_title(2, 'Rechtsgebied')
accordion.set_title(3, 'Instantie')
display(accordion)

# Create text input field.
text = widgets.Text()
display(text)

# Event when button is pressed.
def btn_eventhandler(obj):
    
    years = sliderYear.value
    months = sliderMonth.value
    
    sel2 = selector2.value
    if not sel2:
        sel2 = ["Kamer voor het notariaat in het ressort 's-Hertogenbosch", 'Rechtbank Overijssel', 'Parket bij de Hoge Raad', 'Gerecht in eerste aanleg van Sint Maarten', 'Gerecht in Eerste Aanleg van Aruba', 'College van Beroep voor het bedrijfsleven', 'Gerecht in eerste aanleg van Bonaire, Sint Eustatius en Saba', "Regionaal Tuchtcollege voor de Gezondheidszorg 's-Gravenhage", 'College voor de Rechten van de Mens', 'Gemeenschappelijk Hof van Justitie van Aruba, Curaçao, Sint Maarten en van Bonaire, Sint Eustatius en Saba', 'Kamer voor het notariaat in het ressort Amsterdam', 'Hof van Discipline', 'Rechtbank Gelderland', 'Accountantskamer NIVRA', 'Centraal Tuchtcollege voor de Gezondheidszorg', 'Kamer voor het notariaat in het ressort Arnhem-Leeuwarden', 'Raad van State', 'Rechtbank Zeeland-West-Brabant', "Raad van Discipline 's-Gravenhage", 'Regionaal Tuchtcollege voor de Gezondheidszorg Amsterdam', 'Rechtbank Limburg', 'Gerechtshof Amsterdam', "Raad van Discipline 's-Hertogenbosch", 'Regionaal Tuchtcollege voor de Gezondheidszorg Zwolle', 'Kamer voor het notariaat in het ressort Den Haag', 'Rechtbank Den Haag', 'Rechtbank Midden-Nederland', 'Rechtbank Noord-Nederland', 'Regionaal Tuchtcollege voor de Gezondheidszorg Eindhoven', 'Rechtbank Amsterdam', 'Hoge Raad', 'Kamer voor Gerechtsdeurwaarders', 'Gerecht in eerste aanleg van Curaçao', 'Raad van Discipline Arnhem-Leeuwarden', 'Gerechtshof Arnhem-Leeuwarden', 'Rechtbank Oost-Brabant', "Gerechtshof 's-Hertogenbosch", 'Rechtbank Noord-Holland', 'Raad van Discipline Amsterdam', 'Gerecht in Ambtenarenzaken van Aruba, Curaçao, Sint Maarten en van Bonaire, Sint Eustatius en Saba', 'Rechtbank Rotterdam', 'Regionaal Tuchtcollege voor de Gezondheidszorg Groningen', 'Gerechtshof Den Haag', 'Centrale Raad van Beroep']    
        
    sel1 = selector.value
    if not sel1:
        sel1 = ['Strafrecht; Penitentiair strafrecht', 'Strafrecht; Materieel strafrecht', 'Civiel recht; Personen- en familierecht', 'Strafrecht; Europees strafrecht', 'Bestuursrecht; Bestuursstrafrecht', 'Bestuursrecht; Omgevingsrecht', 'Civiel recht; Insolventierecht', 'Strafrecht', 'Civiel recht; Verbintenissenrecht', 'Bestuursrecht; Socialezekerheidsrecht', 'Bestuursrecht; Ambtenarenrecht', 'Civiel recht; Ondernemingsrecht', 'Bestuursrecht; Vreemdelingenrecht', 'Civiel recht; Internationaal privaatrecht', 'Civiel recht; Aanbestedingsrecht', 'Bestuursrecht; Bestuursprocesrecht', 'Civiel recht; Intellectueel-eigendomsrecht', 'Civiel recht', 'Strafrecht; Internationaal strafrecht', 'Internationaal publiekrecht', 'Civiel recht; Europees civiel recht', 'Bestuursrecht', 'Civiel recht; Goederenrecht', 'Bestuursrecht; Belastingrecht', 'Civiel recht; Burgerlijk procesrecht', None, 'Strafrecht; Strafprocesrecht', 'Civiel recht; Arbeidsrecht']
    
    body = {
  "query": { 
    "bool": { 
      "must": [
        { "match": { "instantie": "{}".format(sel2)}},
        { "match": { "inhoud": "{}".format(text.value)}},
        { "match": { "rechtsgebied": "{}".format(sel1)}},
        { "range" : { "year" : {"gte" : years[0], "lte": years[1]}}},
        { "range" : { "month" : {"gte" : months[0], "lte": months[1]}}}
      ]
    }
  },
  "aggs" : {
    "cloud" : {
        "terms" : { "field" : "inhoud.keyword" }
    }      
  }
}

    results = e.search(index = 'rechtzaken', body = body)
    for hit in results['hits']['hits']:
        # display wordcloud
        wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(hit['_source']['inhoud'])
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()
        # display url
        get_url(hit['_source']['ecli'], hit['_source']['ecli'])
        # display inhoudsindicatie
        print(hit['_source']['inhoud'])
        print('\n\n')

# create button
btn = widgets.Button(description='bing')
btn.style.button_color = 'orange'
display(btn)

# handle button press
btn.on_click(btn_eventhandler)

NameError: name 'widgets' is not defined

# Dataset

The dataset that we used is the open data from a website called Rechtspraak.nl. Rechtspraak.nl contains all descriptions of Dutch court cases dating from 1913 till 2019. 

The first dataset contains the texts from these court cases and the metadata that goes with it, such as the keywords related to it, the publication date, the date of the court case, the court it took place in and the jurisdiction. 

The second dataset is a dictionary with all keywords related to law as key and all the court cases that contain this keyword as values. 

# Assignment
## Requirement 1

> Below we search the woord 'wet'. The result is the top ten hits as determined by Elasticsearch. The title of the hits are links that redirect the user to the full court case on the website rechtspraak.nl. The top ten hits are accompanied by a short description of the court case and a word cloud. Our initial plan was to make the words in the description clickable, so that the user would be redirected to other court cases containing these keywords. Due to time constraints and a lack of understanding of Elasticsearch, we did not manage to implement this. 

![title](imgs/requirement1.png)

## Requirement 2

> You can use the facets to get results from different time periods by moving the slider. There is a slider to select the years and a slider to select the months. We did not manage to build this functionality into the search bar itself. 

![title](imgs/requirement2.png)

## Requirement 3

> We chose to implement option B. Every article has a wordcloud that goes with it, which is created from the 'indhoudsindicatie' (or content indication, in English). A list has been created containing the most common Dutch stopwords to prevent them from showing up in the worldclouds. In the search the wordclouds for the first 10 hits are displayed. 

![title](imgs/requirement3.png)

## Requirement 4

> The timeline has not been implemented yet.

![title](imgs/requirement4.png)

## Requirement 5

> With the different facets the search results can be filtered by year, month, 'rechtsgebied' and 'instantie'. The year and month facets are sliders, with which a certain time period can be selected. We have chosen to seperate year and month, so that if a person only has a preference for certain years and not for months, they won't have to select any months and vice versa. The year slider ranges from 1913 and 2019, because the court cases date from 1913 and 2019.
The facets fro 'rechtsgebied' and 'instantie' are selectors. The user can also select multiple options from these. 


![title](imgs/requirement5.png)

![title](imgs/requirement5b.png)

## Requirement 6

"""5 example queries for our search engine"""


<topic number="1"  >
    <query>*tuchtrecht*</query>
    <description>Find court cases that have something to do with disciplinary law
    </description>

</topic>

<topic number="2"  >
    <query>pinpas fraude </query>
    <description>Find court cases about debit card fraud
    </description>
</topic>

<topic number="3"  >
    <query>diefstal in amsterdam</query>
    <description>Find court cases about theft taking place in amsterdam
    </description>

</topic>

<topic number="4"  >
    <query>fiscale recht kwijtgescholden</query>
    <description> I am looking for cases of fiscal law in which money has been remitted. 
    </description>
</topic>

<topic number="5"  >
    <query>gevangenisstraf van drie maanden</query>
    <description>Find court cases in which the delinquent has to serve three months of jail time
    </description>

</topic>


This is what the final search engine looks like

![title](imgs/requirement6.png)